In [ ]:
import torch
import torchvision
from torchvision.transforms import ToTensor, Normalize, Compose
from torchvision.datasets import MNIST

mnist = MNIST(root='data', 
              train=True, 
              download=True,
              transform=Compose([ToTensor(), Normalize(mean=(0.5,), std=(0.5,))]))

In [ ]:
mnist_test = MNIST(root='data', 
              train=False, 
              download=True,
              transform=Compose([ToTensor(), Normalize(mean=(0.5,), std=(0.5,))]))

In [ ]:
from megaseg.data import MegapixelMNIST 

In [ ]:
training = MegapixelMNIST(
    train=True
)

In [ ]:
data = sparsify(training)
np.save(path.join(args.output_directory, "train.npy"), data)

# Write the test set
test = MegapixelMNIST(
    N=args.n_test,
    train=False,
    W=args.width,
    H=args.height,
    scale=args.scale,
    noise=args.noise,
    seed=args.dataset_seed
)
data = sparsify(test)
np.save(path.join(args.output_directory, "test.npy"), data)

In [ ]:
img, label = mnist[0]
print('Label: ', label)
print(img[:,10:15,10:15])
torch.min(img), torch.max(img)

In [ ]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

img_norm = denorm(img)
plt.imshow(img_norm[0], cmap='gray')
print('Label:', label)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 100
data_loader = DataLoader(mnist, batch_size, shuffle=True)

In [ ]:
for img_batch, label_batch in data_loader:
    print('first batch')
    print(img_batch.shape)
    plt.imshow(img_batch[0][0], cmap='gray')
    print(label_batch)
    break

In [ ]:
mnist

In [ ]:
mnist.train_data.shape